In [2]:
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
from myUtils import *

# rootPath = "../../results/scenario/geothermal"
# rootPath = "../../results/scenario/biogas"
# rootPath = "../../results/scenario/battery"
rootPath = "../../results/scenario/PHS"



plt.style.use('bmh')
%matplotlib inline

nBase = pypsa.Network("../../results/base/base27Node2045/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")
nMean = pypsa.Network(rootPath + "-mean/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")
nOptimist = pypsa.Network(rootPath + "-optimist/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")
nBest = pypsa.Network(rootPath + "/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")


INFO:pypsa.io:Imported network base_s_27_lvopt_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_lvopt_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_lvopt_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_lvopt_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [3]:
import myUtils as utils
import importlib
importlib.reload(utils)

include_biogas = False
include_Geothermie = False
include_warm = False
include_phs = True
include_battery = False
include_dsm = False



def getDfValue(n):
  lists =  [
    # totla 3
    n.statistics.capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.opex(groupby="de_grouper").sum() / 1e9,
    # elec 10
    n.statistics.capex(groupby="de_generator_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_generator_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_generator_grouper").sum() / 1e9,
    n.statistics.optimal_capacity(groupby='de_generator_grouper').sum() / 1e3,
    n.statistics.supply(groupby='de_generator_grouper').sum() / 1e6,
    n.statistics.curtailment(groupby="de_generator_grouper").sum() / 1e6,
    n.statistics.optimal_capacity(groupby="de_import_elec_grouper").sum() / 1e3,
    n.statistics.supply(groupby="de_import_elec_grouper").sum() / 1e6,
    n.statistics.optimal_capacity(groupby="de_export_elec_grouper").sum() / 1e3,
    n.statistics.supply(groupby="de_export_elec_grouper").sum() / 1e6,    
  ]

  if include_warm:
    lists.extend([
      n.statistics.capex(groupby="de_heat_generator_grouper").sum() / 1e9,
      n.statistics.expanded_capex(groupby="de_heat_generator_grouper").sum() / 1e9,
      n.statistics.opex(groupby="de_heat_generator_grouper").sum() / 1e9,
      n.statistics.optimal_capacity(groupby='de_heat_generator_grouper').sum() / 1e3,
      n.statistics.supply(groupby="de_heat_generator_grouper").sum() / 1e6,
    ])

  if include_Geothermie:
    lists.extend([
      n.statistics.supply(groupby=["carrier", "de_generator_grouper"]).groupby('carrier').sum().get('geothermal organic rankine cycle', 0),
      renameHeatCarrier(n.statistics.supply(groupby=["carrier", "de_heat_generator_grouper"]).groupby('carrier').sum()).get('Geothermie', 0),
    ])
  
  if include_biogas:
    lists.extend([
      n.statistics.supply(groupby="de_biogas_generator_grouper").sum() /1e6,
      n.statistics.supply(groupby="de_gas_generator_grouper").sum() /1e6,
      n.statistics.supply(groupby="de_biogas_generator_grouper").sum()/n.statistics.supply(groupby="de_gas_generator_grouper").sum() * 100
    ])

  if include_phs:
    lists.extend([
      n.statistics.capex(groupby="de_phs_grouper").sum() /1e9,
      n.statistics.optimal_capacity(groupby="de_phs_grouper").sum() / 1e3,
      n.statistics.supply(groupby="de_phs_grouper").sum() /1e6,
    ])

  if include_battery:

    batteryStoreIds = utils.de_battery_store_grouper(n, 'Store').tolist()
    n.stores_t.p[batteryStoreIds].clip(upper=0).sum()
    
    lists.extend([
      n.statistics.capex(groupby="de_battery_store_grouper").sum(),
      n.statistics.optimal_capacity(groupby="de_battery_store_grouper").sum(),
      # n.statistics.supply(groupby="de_battery_store_grouper").sum(),
      - n.stores_t.p[batteryStoreIds].clip(upper=0).sum().sum(),
    ])
  
  if include_dsm:
    dsmStoreIds = utils.de_dsm_grouper(n, 'Store').tolist()
    dsmLinkIds =  utils.de_dsm_grouper(n, 'Link').tolist()

    lists.extend([
      - n.stores_t.p[dsmStoreIds].sum().sum(),
      - n.links_t.p1[dsmLinkIds].sum().sum()
    ])

  
  return lists

data = {
    'Name': [
       # total
      'CAPEX', 
      'investierte CAPEX', 
      'OPEX',
       # elec
      'Strom CAPEX',
      'Strom investierte CAPEX',
      'Strom OPEX',
      'Stromkapazität',
      'Stromerzeugung',
      'curtailment',
      'Importkapazität',
      'Importmenge',
      'Exportkapazität',
      'Exportmenge',
      ],
    'Unit': [
      # total
      'Mrd.EUR',
      'Mrd.EUR',
      'Mrd.EUR',
       # elec
      'Mrd.EUR',
      'Mrd.EUR',
      'Mrd.EUR',
      'GW',
      'TWH',
      'TWH',
      'GW',
      'TWH',
      'GW',
      'TWH',      
    ],
    'Base': getDfValue(nBase),
    'Mean': getDfValue(nMean),
    'Optimist': getDfValue(nOptimist),
    'Best': getDfValue(nBest)
}

if include_warm:
  data['Name'].extend([
    'Wärm CAPEX',
    'Wärm investierte CAPEX',
    'Wärm OPEX',
    'Wärmekapazität',
    'Wärmerzeugung',
  ])
  data['Unit'].extend([
    'Mrd.EUR',
    'Mrd.EUR',
    'Mrd.EUR',
    'GW',
    'TWH',     
  ])

if include_Geothermie:
  data['Name'].extend([
    'geothermal elec menge',
    'geothermal Wärm menge'
  ])
  data['Unit'].extend([
    'MWH',
    'MWH',
  ])

if include_biogas:
  data['Name'].extend([
    'Biogasproduktion',
    'Gasproduktion',
    'biogas/Gas'
  ])
  data['Unit'].extend([
    'MWH',
    'MWH',
    '%'
  ])
if include_phs:
  data['Name'].extend([
    'PHS CAPEX',
    'PHS Stromkapazität',
    'PHS Stromerzeugung'
  ])
  data['Unit'].extend([
    'Mrd.EUR',
    'GW',
    'TWH'
  ])

if include_battery:
  data['Name'].extend([
    'Battery CAPEX',
    'Battery Stromkapazität',
    'Battery Stromerzeugung'
  ])
  data['Unit'].extend([
    'EUR',
    'MW',
    'MWH'
  ])
if include_dsm:
  data['Name'].extend([
    'DSM Stromerzeugung',
    'V2G Stromerzeugung'
  ])
  data['Unit'].extend([
    'MWH',
    'MWH'
  ])

print(len(data['Name']), len(data['Unit']), len(data['Base']))

df = pd.DataFrame(data).set_index('Name')
table = dataframe_to_table(df)
print(table)

16 16 16
  [CAPEX], [Mrd.EUR], [64.255], [64.113], [63.966], [63.524],
  [investierte CAPEX], [Mrd.EUR], [44.917], [44.929], [44.933], [45.561],
  [OPEX], [Mrd.EUR], [6.631], [6.635], [6.677], [6.656],
  [Strom CAPEX], [Mrd.EUR], [26.205], [26.212], [26.230], [26.365],
  [Strom investierte CAPEX], [Mrd.EUR], [10.906], [10.913], [10.930], [11.066],
  [Strom OPEX], [Mrd.EUR], [10.906], [10.913], [10.930], [11.066],
  [Stromkapazität], [GW], [210.524], [210.651], [210.958], [213.343],
  [Stromerzeugung], [TWH], [589.774], [592.704], [592.170], [591.334],
  [curtailment], [TWH], [7.862], [5.053], [5.878], [8.981],
  [Importkapazität], [GW], [92.970], [92.778], [92.624], [94.671],
  [Importmenge], [TWH], [269.297], [267.933], [267.284], [266.069],
  [Exportkapazität], [GW], [98.193], [97.904], [97.620], [101.464],
  [Exportmenge], [TWH], [243.774], [243.719], [243.681], [240.694],
  [PHS CAPEX], [Mrd.EUR], [1.375], [1.222], [1.070], [0.000],
  [PHS Stromkapazität], [GW], [7.526], [7.526], [

```
1 kW= 1e3 W               1 KWH = 1e3 WH             1 Tsd EUR = 1e3 EUR
1 MW = 1e6 W = 1 MW       1 MWH = 1e6 WH = 1 MWH     1 Mio.EUR = 1e6 EUR
1 GW = 1e9 W = 1e3 MW     1 GWH = 1e9 WH = 1e3 MWH   1 Mrd.EUR = 1e9 EUR
1 TW = 1e12 W = 1e6 MW    1 TWH = 1e12 WH = 1e6 MWH  1 Bio.EUR = 1e12 EUR


```